## Predicting the duration of a ride, using linear regression. Train on Jan '21 data, validate with Feb '21.

Week 2 Homework for MLops-zoomcamp, DataTalks.Club https://github.com/DataTalksClub/mlops-zoomcamp

Code created by Joshua Harvey, 2022
joshuasharvey@gmail.com

https://github.com/hirschland

Running on local machine

Data from NYC Trip records, https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

# Q1. Install MLflow

In [4]:
!conda env list

# conda environments:
#
regnmf                   /Users/harvej05/.conda/envs/regnmf
base                     /opt/anaconda3
CaImAn                   /opt/anaconda3/envs/CaImAn
caiman                   /opt/anaconda3/envs/caiman
cellpose                 /opt/anaconda3/envs/cellpose
exp-tracking          *  /opt/anaconda3/envs/exp-tracking
hmm-env                  /opt/anaconda3/envs/hmm-env
iML                      /opt/anaconda3/envs/iML
locanmf                  /opt/anaconda3/envs/locanmf
suite2p                  /opt/anaconda3/envs/suite2p



### Working in exp-tracking virtual environment

In [5]:
!mlflow --version

mlflow, version 1.26.0


### With version 1.26.0 of mlflow

# Q2. Download and preprocess the data

In [13]:
# download data
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet
%cd ..

/Users/harvej05/Dropbox (Personal)/My Mac (HARVEJ05-MBP16T)/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/data
--2022-06-01 20:09:15--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.172.184
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.172.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘green_tripdata_2021-01.parquet’

green_tripdata_2021 100%[===================>]   1.27M  --.-KB/s    in 0.1s    

2022-06-01 20:09:16 (9.71 MB/s) - ‘green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-06-01 20:09:16--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.172.184
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.172.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M

In [15]:
# preprocess data
!python /Users/harvej05/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/preprocess_data.py --raw_data_path /Users/harvej05/Dropbox (Personal)/My Mac (HARVEJ05-MBP16T)/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/data --dest_path ./output

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `python /Users/harvej05/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/preprocess_data.py --raw_data_path /Users/harvej05/Dropbox (Personal)/My Mac (HARVEJ05-MBP16T)/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/data --dest_path ./output'


### 4 files were saved to the output folder: dv.pkl, test.pkl, train.pkl, valid.pkl

# Q3. Train a model with autolog

In [16]:
# edit train.py to enable autologging

In [27]:
!python /Users/harvej05/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/train.py

2022/06/01 20:18:48 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi' does not exist. Creating a new experiment.
2022/06/01 20:18:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
(Experiment)
    experiment_id=0
    name='Default'
    artifact_location='artifacts/0'

(Experiment)
    experiment_id=4
    name='nyc-taxi'
    artifact_location='./mlruns/4'

2022/06/01 20:19:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


### There are 17 parameters logged by MLflow

# Q4. Launch the tracking server locally

In [ ]:
#mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

# Q5. Tune the hyperparameters of the model


In [38]:
!python /Users/harvej05/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/hpo.py

100%|█████████| 50/50 [06:41<00:00,  8.02s/trial, best loss: 6.6284257482044735]


### Best validation RMSE is 6.628, for max depth=19, min_samples_leaf=3, min_samples_split=5

# Q6. Promote the best model to the model registry


In [39]:
!python /Users/harvej05/Documents/MLOps/mlops-zoomcamp/02-experiment-tracking/homework/register_model.py

2022/06/01 21:12:19 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/06/01 21:12:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Registered model 'nyc-taxi-promoted' already exists. Creating a new version of this model...
2022/06/01 21:13:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-promoted, version 2
Created version '2' of model 'nyc-taxi-promoted'.


### RMSE of best model is 6.5